In [30]:
import pandas as pd
import numpy as np
import numpy as np
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.preprocessing import Imputer
from sklearn.base import TransformerMixin
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from imblearn.metrics import classification_report_imbalanced

In [31]:
#Reading data files
path = r'C:\Users\az9826\Desktop\Machine_learning'
filename_sen1 = path+'\\'+'Sen_bill_status_contemporary.json'
filename_sen2 = path+'\\'+'Sen_bill_status_modern.json'
f_sm = path+'\\'+'Majority Party Chairmanships - Senate.csv'
f_lc = path+'\\'+'legislators-current.csv'
f_lh = path+'\\'+'legislators-historical.csv'

In [32]:
#Getting the details about the congress and which party was in majority for that bill
df_sm = pd.read_csv(f_sm)
df_sm = df_sm[['Congress', 'MajParty']]
df_sm = df_sm.drop_duplicates()

In [33]:
#Data preprocessing : fetching sponsors name and to which party they belong from legislators historical table
#There was a seperate fiel for full name in the data but the data was ambegiuous, therefore create the full name using the 
#other two fields present that is first name and last name
#keeping only those records where party type is sen taht is senate 
#And keeping only those records where party is either Republican or Democrat sinced there were many other parties as well 
# but these two are of our interest 
df_lc = pd.read_csv(f_lc)
df_lh = pd.read_csv(f_lh)
df_lh['full_name'] = df_lh['last_name'].fillna('')+', '+df_lh['first_name'].fillna('')
df_lh = df_lh[df_lh['type'] == 'sen']
df_lh = df_lh[['full_name', 'party']]
party = ["Republican", "Democrat"]
df_lh = df_lh[df_lh['party'].isin(party)]

In [34]:
#Appending the two senate files because we have collected the data from two different sources
df1 = pd.read_json(filename_sen1)
df2 = pd.read_json(filename_sen2)
df = df1.append(df2, ignore_index=True)
df = df.fillna(np.nan)


C:\Users\az9826\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


In [35]:
# normalizing the data by creating flags 
#if there is any data in the cosponsor field taht is any non missing value make the flag 1 else 0 
#if there is non missing value in the field amendments make the flag 1 else 0
#if there is non missing value in the field  popular_title make the flag 1 else 0
#if there is non missing value in the field  related_bills make the flag 1 else 0
# creating full name for the sponsor , keeping only first and last name
df['cosponsor_flg'] = np.where(df['cosponsors'].apply(len)==0, 0, 1)
df['amendments_flg'] = np.where(df['amendments'].apply(len)==0, 0, 1)
df['popular_flg'] = np.where(df['popular_title'].isnull(), 0, 1)
df['related_bills_flg'] = np.where(df['related_bills'].apply(len)==0, 0, 1)
df['sponsor_nm'] = [d.get('name') for d in df.sponsor]
df['full_name'] = df['sponsor_nm'].str.split(' ',expand = True)[0]+' '+df['sponsor_nm'].str.split(' ',expand = True)[1] 
df['Congress'] = pd.to_numeric(df['congress'], errors='coerce')

In [36]:
#merging files on sponsor names and congress
#keeping only those records where status of the bill is either passed or enacted
df_sen1 = pd.merge(df, df_lh, how='inner', on=['full_name'])
df_sen2 = pd.merge(df_sen1, df_sm, how='inner', on=['Congress'])
df_sen3 = df_sen2[df_sen2['party'].isin(party)]
df_sen3['party_flg'] = np.where(df_sen3['party'] == "Republican", 0, 1)
df_sen3['majority_flg'] = np.where(df_sen3['MajParty'] == "R", 0, 1)
statuses = ["PASSED:BILL", "ENACTED:SIGNED", "REPORTED", "PROV_KILL:SUSPENSIONFAILED", "PROV_KILL:CLOTUREFAILED",
           "FAIL:ORIGINATING:SENATE", "PASSED:SIMPLERES", "PASSED:CONSTAMEND", "PASS_OVER:SENATE", "PASSED:CONCURRENTRES",
           "FAIL:SECOND:SENATE", "PASS_BACK:SENATE", "PROV_KILL:PINGPONGFAIL", "CONFERENCE:PASSED:SENATE", "PROV_KILL:VETO",
           "VETOED:POCKET","VETOED:OVERRIDE_FAIL_ORIGINATING:SENATE","VETOED:OVERRIDE_PASS_OVER:SENATE", "VETOED:OVERRIDE_FAIL_SECOND:SENATE",
           "ENACTED:VETO_OVERRIDE", "ENACTED:TENDAYRULE"]
df_sen3['Class'] = np.where(df_sen3['status'].isin(statuses), 1, 0)

In [37]:
#our class field is Status
df_sen = df_sen3[['bill_id', 'cosponsor_flg','amendments_flg', 'popular_flg', 'related_bills_flg', 'party_flg', 'majority_flg', 'Class']]
df_sen = df_sen.reset_index(drop = True)


In [38]:
#Final Feature selection
#taking 80% training data
features = list(df_sen.columns[1:7])
traindatasize = int(len(df_sen)*.8)
testdatasize = len(df_sen) - traindatasize


In [39]:
#assigning data for tarining and test
traindata = df_sen[0:traindatasize]
testdata = df_sen[traindatasize:len(df_sen)+1]

df2 = traindata
y_train = df2["Class"]
X_train = df2[features]

df3 = testdata
y_test = df3["Class"]
X_test = df3[features]


In [40]:
#applying random forest 
rf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
rf.fit(X_train, y_train)
y_predicted_rf = rf.predict(X_test)
y_score_rf = rf.score(X_test,y_test)
y_predicted_rf
#y_score_rf

array([0, 0, 0, ..., 0, 0, 0])

In [41]:
y_score_rf

0.886372510737993

In [42]:
#applying decision tree
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
y_predicted = dt.predict(X_test)
y_score = dt.score(X_test,y_test)
y_predicted

array([0, 0, 0, ..., 0, 0, 0])

In [43]:
y_score

0.8859820382663022

In [44]:
#applying SVM
d_svm = svm.SVC()
d_svm.fit(X_train, y_train)
y_predicted_svm = d_svm.predict(X_test)
y_score_svm = d_svm.score(X_test,y_test)
y_predicted_svm

array([0, 0, 0, ..., 0, 0, 0])

In [45]:
y_score_svm

0.8855915657946115

In [46]:
print(classification_report_imbalanced(y_test, y_predicted_rf))


                   pre       rec       spe        f1       geo       iba       sup

          0       0.88      1.00      0.16      0.94      0.92      0.83      2217
          1       0.95      0.16      1.00      0.28      0.92      0.85       344

avg / total       0.89      0.89      0.28      0.85      0.92      0.84      2561



In [49]:
#converting data into csv for visualization 
df_sen.to_csv(path+'senate_viz.csv')